In [33]:
from dimod import (
    Binary,
    BinaryQuadraticModel,
    ConstrainedQuadraticModel,
    quicksum,
)

from dwave.system import LeapHybridCQMSampler

import random
import pandas as pd

In [34]:
def build_vrp_cqm(num_locations, distances, num_vehicles):
    """
    Input:
        num_locations: number of distinct locations for orders
        distances: an matrix with the distances between respective locations for a fully connected graph
        num_vehicles: the number of vehicles used
    """

    M = num_vehicles
    N = num_locations

    cqm = ConstrainedQuadraticModel()

    # Create all the variables: one for each vehicle/location/position combo
    # k is position, j is vertex, i is vehicle
    x = {(i, j, k): Binary(str(i) + "_" + str(j) + "_" + str(k)) for k in range(N+1) for j in range(N+1) for i in range(M)}

    print(f"{x}\n")

    # Define the unconstrained binary optimization problem
    obj = BinaryQuadraticModel(vartype="BINARY")

    # The cost of going from the depot to the first stop
    for m in range(M):
        for n in range(N):
            obj += x[m, n, 1] * distances[N][n]

    # The cost of going from the last stop to the depot in the case of one vehicle (vanishes otherwise)
    for m in range(M):
        for n in range(N):
            obj += x[m, n, N] * distances[N][n]

    # The cost of going from the first stop to all others and back to the depot
    for m in range(M):
        for n in range(N-1):
            for i in range(N+1):
                for j in range(N+1):
                    obj += x[m, i, n] * x[m, j, n] * distances[i][j]

    cqm.set_objective(obj)

    # Implement constraints:

    # 1. Each location should be served by exactly one vehicle
    for j in range(N):

        sum = 0

        for m in range(M):
            for k in range(N+1):
                sum += x[m, j, k]

        cqm.add_constraint(sum == 1,
                           label=f"Vertex {j} is not visited or visited more than once")

    # 2. Each vehicle is in one place at one time
    for i in range(M):
        for k in range(N+1):

            sum = 0

            for j in range(N + 1):
                sum += x[i, j, k]
            
            cqm.add_constraint(sum == 1,
                               label=f"Vehicle {i} is at more or less than one position at time {k}")
            
    # Return the constrained optimization solution
    return cqm
            

def run_cqm(cqm):
    """Run the provided CQM on the Leap Hybrid CQM Sampler."""
    sampler = LeapHybridCQMSampler(token="DEV-7a13065bb40813db4f53d6af8d005e3598588034")

    sampleset = sampler.sample_cqm(cqm)
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    num_feasible = len(feasible_sampleset)
    errors = " "
    if num_feasible == 0:
        msg = "\nNo feasible solution found.\n"
        errors += msg
        print(msg)
        sat_array = sampleset.first.is_satisfied

        # if sampleset is all 0's, set at least one variable to 1 
        # --> this is needed for "Solve CQM" button to function propertly
        s_vals = set(sampleset.first.sample.values())
        if s_vals == {0.0}:
            sampleset.first.sample[list(cqm.variables)[0]] = 1.0

        constraints = sampleset.info["constraint_labels"]

        print(constraints)
       
        return sampleset, errors

    print("\nFeasible solution found.\n")

    return feasible_sampleset, None

In [35]:
# Create a sample problem
num_destinations = 10
num_vehicles = 2

cost_matrix = []

for i in range(num_destinations + 1):
    row = []
    for j in range(num_destinations + 1):
        if i == j:
            row.append(0)
        else:
            row.append(random.randint(2, 5))

    cost_matrix.append(row)


cqm = build_vrp_cqm(num_destinations, cost_matrix, num_vehicles)

{(0, 0, 0): BinaryQuadraticModel({'0_0_0': 1.0}, {}, 0.0, 'BINARY'), (1, 0, 0): BinaryQuadraticModel({'1_0_0': 1.0}, {}, 0.0, 'BINARY'), (0, 1, 0): BinaryQuadraticModel({'0_1_0': 1.0}, {}, 0.0, 'BINARY'), (1, 1, 0): BinaryQuadraticModel({'1_1_0': 1.0}, {}, 0.0, 'BINARY'), (0, 2, 0): BinaryQuadraticModel({'0_2_0': 1.0}, {}, 0.0, 'BINARY'), (1, 2, 0): BinaryQuadraticModel({'1_2_0': 1.0}, {}, 0.0, 'BINARY'), (0, 3, 0): BinaryQuadraticModel({'0_3_0': 1.0}, {}, 0.0, 'BINARY'), (1, 3, 0): BinaryQuadraticModel({'1_3_0': 1.0}, {}, 0.0, 'BINARY'), (0, 4, 0): BinaryQuadraticModel({'0_4_0': 1.0}, {}, 0.0, 'BINARY'), (1, 4, 0): BinaryQuadraticModel({'1_4_0': 1.0}, {}, 0.0, 'BINARY'), (0, 5, 0): BinaryQuadraticModel({'0_5_0': 1.0}, {}, 0.0, 'BINARY'), (1, 5, 0): BinaryQuadraticModel({'1_5_0': 1.0}, {}, 0.0, 'BINARY'), (0, 6, 0): BinaryQuadraticModel({'0_6_0': 1.0}, {}, 0.0, 'BINARY'), (1, 6, 0): BinaryQuadraticModel({'1_6_0': 1.0}, {}, 0.0, 'BINARY'), (0, 7, 0): BinaryQuadraticModel({'0_7_0': 1.0},

In [36]:
feasible_sampleset, errors = run_cqm(cqm)


Feasible solution found.



In [43]:
def parse_string(input_string):
    return list(map(int, input_string.split('_')))

def build_routes_from_sample(sample, num_vehicles, num_destinations):
    """Builds a set of routes from the sample returned."""

    routes =  [0]*num_vehicles

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            routes[vehicle].append(vertex)

    return routes

In [44]:
print(feasible_sampleset)

sample = feasible_sampleset.first.sample

print(sample)

routes = build_routes_from_sample(sample, num_vehicles, num_destinations)

print(routes)

   0_0_0 0_0_1 0_0_10 0_0_2 0_0_3 0_0_4 0_0_5 ... 1_9_9 energy num_oc. ...
0    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   1.0    0.0       1 ...
1    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
2    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
3    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
4    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
5    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
6    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
7    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   1.0    0.0       1 ...
8    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
9    0.0   0.0    0.0   0.0   0.0   0.0   1.0 ...   0.0    0.0       1 ...
10   0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
11   0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0    0.0       1 ...
12   0.0   0.0    0.0   0

AttributeError: 'int' object has no attribute 'append'